# Question Answering using Embeddings

Many use cases require GPT-3 to respond to user questions with insightful answers. For example, a customer support chatbot may need to provide answers to common questions. The GPT models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate a method for enabling GPT-3 to answer questions using a library of text as a reference, by using document embeddings and retrieval. We'll be using a dataset of Wikipedia articles about the 2020 Summer Olympic Games. Please see [this notebook](fine-tuned_qa/olympics-1-collect-data.ipynb) to follow the data gathering process.

In [1]:
import numpy as np
import pandas as pd
import openai
import pickle
import tiktoken
from transformers import GPT2TokenizerFast

# TODO:
# Configure your OpenAI API key and select the appropriate models

# COMPLETIONS_MODEL = "Used for text generation"
# EMBEDDING_MODEL = "Used for generating embeddings"

By default, GPT-3 isn't an expert on the 2020 Olympics:

In [2]:
prompt = "Who won the 2020 Summer Olympics men's high jump?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Marcelo Chierighini of Brazil won the gold medal in the men's high jump at the 2020 Summer Olympics."

Marcelo is a gold medalist swimmer, and, we assume, not much of a high jumper! Evidently GPT-3 needs some assistance here. 

The first issue to tackle is that the model is hallucinating an answer rather than telling us "I don't know". This is bad because it makes it hard to trust the answer that the model gives us! 

# 0) Preventing hallucination with prompt engineering

We can address this hallucination issue by being more explicit with our prompt:


In [3]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Sorry, I don't know."

To help the model answer the question, we provide extra contextual information in the prompt. When the total required context is short, we can include it in the prompt directly. For example we can use this information taken from Wikipedia. We update the initial prompt to tell the model to explicitly make use of the provided text.

In [4]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Gianmarco Tamberi and Mutaz Essa Barshim emerged as joint winners of the event.'

Adding extra information into the prompt only works when the dataset of extra content that the model may need to know is small enough to fit in a single prompt. What do we do when we need the model to choose relevant contextual information from within a large body of information?

**In the remainder of this notebook, we will demonstrate a method for augmenting GPT-3 with a large body of additional contextual information by using document embeddings and retrieval.** This method answers queries in two steps: first it retrieves the information relevant to the query, then it writes an answer tailored to the question based on the retrieved information. The first step uses the [Embeddings API](https://beta.openai.com/docs/guides/embeddings), the second step uses the [Completions API](https://beta.openai.com/docs/guides/completion/introduction).
 
The steps are:
* Preprocess the contextual information by splitting it into chunks and create an embedding vector for each chunk.
* On receiving a query, embed the query in the same vector space as the context chunks and find the context embeddings which are most similar to the query.
* Prepend the most relevant context embeddings to the query prompt.
* Submit the question along with the most relevant context to GPT, and receive an answer which makes use of the provided contextual information.

# 1) Preprocess the document library

We plan to use document embeddings to fetch the most relevant part of parts of our document library and insert them into the prompt that we provide to GPT-3. We therefore need to break up the document library into "sections" of context, which can be searched and retrieved separately. 

Sections should be large enough to contain enough information to answer a question; but small enough to fit one or several into the GPT-3 prompt. We find that approximately a paragraph of text is usually a good length, but you should experiment for your particular use case. In this example, Wikipedia articles are already grouped into semantically related headers, so we will use these to define our sections. This preprocessing has already been done in [this notebook](fine-tuned_qa/olympics-1-collect-data.ipynb), so we will load the results and use them.

In [5]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('./data/Series4000.csv', header=0)
# df = df.set_index(["title", "heading"])
# df = df.set_axis(['title', 'heading'] + list(df.columns[2:]), axis=1)
print(f"{len(df)} rows in the data.")
df.sample(5)

9 rows in the data.


,title,heading1,heading2,heading3,heading4,content
7,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(c) Mortgage instruments for ARMs - Instructio...,"The Seller must complete Section 4(D), Limits ..."
2,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.1 The Mortgage application,(c) Electronic and fax copies of loan applicat...,Freddie Mac agrees that the Seller may receive...
6,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(c) Mortgage instruments for ARMs - Use of Fan...,The Seller may use Fannie Mae's ARM instrument...
3,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(a) Use of Uniform Instruments,The Security Instrument and Note must be execu...
5,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(c) Mortgage instruments for ARMs - Required A...,Required ARM Uniform Instruments\nARMs must be...


In [6]:
# TODO:
# Create a new column called "context" that combines multiple fields into a single text string.
#
# Think about:
# - Which columns should be included (e.g., title, headings, content)?
# - How you want to separate different fields (commas, semicolons, new lines, labels, etc.).
# - Whether you need to clean the text (e.g., strip extra whitespace).
#
# Try at least one of the following strategies:
# 1. String concatenation using "+" operators
# 2. Using Python f-strings
# 3. Applying a function row-wise with df.apply(...)
#
# Goal:
# Produce a single text field that can later be used for tasks like search, embeddings, or retrieval.

,title,heading1,heading2,heading3,heading4,content,context
0,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.1 The Mortgage application,"(a) Required use of Form 65, Uniform Residenti...","Form 65, Uniform Residential Loan Application,...",title: Series 4000: Mortgage Eligibility; head...
1,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.1 The Mortgage application,(b) Completion instructions,A completed Form 65 is used to begin the proce...,title: Series 4000: Mortgage Eligibility; head...


## 1.1 Create some questions based on context

In [7]:
import openai

def get_questions(context):
    try:
        response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"Write questions based on the text below\n\nText: {context}\n\nQuestions:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response['choices'][0]['text']
    except:
        return ""


df['questions']= df.context.apply(get_questions)
df['questions'] = "1." + df.questions
print(df[['questions']].values[0][0])

1. What is the required use of Form 65?
2. What are the Seller's formatting options for Form 65?
3. What are the translation aids for Form 65?


## 1.2 Create answers based on the context

In [8]:
def get_answers(row):
    try:
        response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"Write answer based on the text below\n\nText: {row.context}\n\nQuestions:\n{row.questions}\n\nAnswers:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response['choices'][0]['text']
    except Exception as e:
        print (e)
        return ""


df['answers']= df.apply(get_answers, axis=1)
df['answers'] = "1." + df.answers
df = df.dropna().reset_index().drop('index',axis=1)
print(df[['answers']].values[0][0])

1. The required use of Form 65 is for all Mortgage applications.
2. The Seller has formatting options for Form 65, which must be in accordance with the UMDP Rendering Options.
3. The translation aids for Form 65 are for reference only and are not to be executed.


In [9]:
def num_tokens(text: str, model: str = "gpt-4") -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

df['tokens'] = df.apply(lambda row : num_tokens(row['context']), axis = 1)

In [10]:
df.to_csv('data/selling_qa.csv', index=False)

We preprocess the document sections by creating an embedding vector for each section. An embedding is a vector of numbers that helps us understand how semantically similar or different the texts are. The closer two embeddings are to each other, the more similar are their contents. See the [documentation on OpenAI embeddings](https://beta.openai.com/docs/guides/embeddings) for more information.

This indexing stage can be executed offline and only runs once to precompute the indexes for the dataset so that each piece of content can be retrieved later. Since this is a small example, we will store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/) or [Weaviate](https://github.com/semi-technologies/weaviate) to power the search.

In [11]:
# TODO:
# Implement a function that converts a piece of text into an embedding vector
# using the OpenAI Embeddings API.
#
# Think about:
# - Which embedding model to use
# - What parameters the API expects
# - How to extract the embedding from the API response
#
# Function signature should remain the same.
def get_embedding(text: str, model: str = EMBEDDING_MODEL):
    pass


# TODO:
# Implement a function that creates embeddings for each row in a DataFrame.
#
# Requirements:
# - Generate one embedding per document/row
# - Associate each embedding with the row index
#
# Think about:
# - Iterating with df.iterrows(), df.itertuples(), or vectorized approaches
# - Whether batching API calls would be beneficial
# - How this approach would scale for large datasets
#
# Return:
# A dictionary mapping row index -> embedding vector
def compute_doc_embeddings(df: pd.DataFrame):
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    """
    pass

In [12]:
document_embeddings = compute_doc_embeddings(df)

In [13]:
# JSON save and load
import json

def jsonKeys2int(x):
    if isinstance(x, dict):
        return {int(k):v for k,v in x.items()}
    return x

with open('./data/embeddings.json', 'w') as fp:
    json.dump(document_embeddings, fp)
    
with open('./data/embeddings.json', 'r') as fp:
    document_embeddings = json.load(fp, object_hook=jsonKeys2int)

In [14]:
def load_embeddings(fname: str):
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.

In [15]:
# document_embeddings = load_embeddings("https://cdn.openai.com/API/examples/data/olympics_sections_document_embeddings.csv")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

# document_embeddings = compute_doc_embeddings(df)

In [16]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

0 : [-0.01663094572722912, -0.0030637693125754595, -0.0005449180025607347, -0.025303363800048828, -0.003906553611159325]... (1536 entries)


So we have split our document library into sections, and encoded them by creating embedding vectors that represent each chunk. Next we will use these embeddings to answer our users' questions.

# 2) Find the most similar document embeddings to the question embedding

At the time of question-answering, to answer the user's query we compute the query embedding of the question and use it to find the most similar document sections. Since this is a small example, we store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/) or [Weaviate](https://github.com/semi-technologies/weaviate) to power the search.

In [17]:
def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query, contexts):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [18]:
order_document_sections_by_query_similarity("Can I use premium financing to fund the down payment?", document_embeddings)[:5]

[(0.7598378308299598, 1),
 (0.7553676268717341, 0),
 (0.7496156955734523, 3),
 (0.748820164292117, 4),
 (0.7480549316545756, 2)]

In [19]:
order_document_sections_by_query_similarity("Can I use premium financing to fund closing costs and prepaids?", document_embeddings)[:5]

[(0.7627212871496409, 1),
 (0.7521531976708908, 4),
 (0.7480105908092212, 2),
 (0.7469793495329591, 0),
 (0.7417589109150609, 6)]

We can see that the most relevant document sections for each question include the summaries for the Men's and Women's high jump competitions - which is exactly what we would expect.

# 3) Add the most relevant document sections to the query prompt

Once we've calculated the most relevant pieces of context, we construct a prompt by simply prepending them to the supplied query. It is helpful to use a query separator to help the model distinguish between separate pieces of text.

In [20]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
separator_len = len(tokenizer.tokenize(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [21]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [22]:
prompt = construct_prompt(
    "What is the required use of Form 65?",
    document_embeddings,
    df
)

print("===\n", prompt)

Selected 1 document sections:
0
===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* Form 65, Uniform Residential Loan Application, must be used for all Mortgage applications.  The Seller must use the version of Form 65 that is current as of the date of the loan application. See Exhibit 4A, Single-Family Uniform Instruments, for the date of the current version of Form 65 and the most current version of the Uniform Mortgage Data Program® (UMDP®) Instructions for Completing the Uniform Residential Loan Application.  Seller may make changes to the style and formatting of the Form 65 and its components – Borrower Information, Additional Borrower, Continuation Sheet, Lender Loan Information and the Unmarried Addendum, if applicable, in accordance with the UMDP Rendering Options for the Uniform Residential Loan Application, Document revised 1/2020(PDF 5mb opens in new window) 

We have now obtained the document sections that are most relevant to the question. As a final step, let's put it all together to get an answer to the question.

# 4) Answer the user's question based on the context.

Now that we've retrieved the relevant context and constructed our prompt, we can finally use the Completions API to answer the user's query.

In [23]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [24]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embedding,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [25]:
answer_query_with_context("What is the required use of Form 65?", df, document_embeddings)

Selected 1 document sections:
0


'Form 65 must be used for all Mortgage applications, and the Seller must use the version of Form 65 that is current as of the date of the loan application. The fields names, descriptions, and order of sections may not be altered in anyway, but Form fields within a section may be moved within that section if additional field length is needed. Any adjustments made to the format of the form must be made pursuant to all applicable law.'

In [26]:
answer_query_with_context("What are the Seller's formatting options for Form 65?", df, document_embeddings)

Selected 1 document sections:
2


'The Seller may receive an initial Form 65 from a Borrower as an Electronic Record or fax copy, and the final loan application delivered by the Borrower to the Seller at loan closing must be an original paper Form 65 if necessary, either physically signed by the Borrower using a pen or signed electronically by the Borrower at closing (settlement) using an Electronic Signature that meets the requirements in Chapter 1401.'

In [27]:
answer_query_with_context("What are the translation aids for Form 65?", df, document_embeddings)

Selected 1 document sections:
0


'The translation aids for Form 65 are available on Freddie Mac’s Multi-language Resources for Lenders and Other Housing Professionals web page. These translation aids complement the applicable English-language documents and may be provided to consumers as supplemental education material when originating single-family residential Mortgages. The translation aids are for reference only and are not to be executed.'

## Test with more examples

In [28]:
query = "The Seller has formatting options for Form 65, which must be in accordance with the UMDP Rendering Options."
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
0

Q: The Seller has formatting options for Form 65, which must be in accordance with the UMDP Rendering Options.
A: Yes, the Seller has formatting options for Form 65, which must be in accordance with the UMDP Rendering Options.
